# Violence Detection using CNN + LSTM neural network

## Imports

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
%cd '/content/gdrive/MyDrive/Practice Module/Violence-Detection'
pwd = os.getcwd()
print(pwd)

Mounted at /content/gdrive
/content/gdrive/MyDrive/Practice Module/Violence-Detection
/content/gdrive/MyDrive/Practice Module/Violence-Detection


In [2]:
%matplotlib inline
import cv2
import os
import glob
import random
import sys
import h5py
import numpy as np
# import matplotlib.pyplot as plt
# from random import shuffle
# from sklearn.metrics import confusion_matrix, accuracy_score

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
from tensorflow.keras.applications import VGG16
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Activation , Dropout


## Helper Functions

We will use the function ```print_progress``` to print the amount of videos processed the datasets

In [3]:
def print_progress(count, max_count):
    # Percentage completion.
    pct_complete = count / max_count

    # Status-message. Note the \r which means the line should overwrite itself.
    msg = "\r- Progress: {0:.1%}".format(pct_complete)

    sys.stdout.write(msg)
    sys.stdout.flush()

## Load Data

Firstly, we define the directory to place the video dataset

Copy some of the data-dimensions for convenience.

In [4]:
# Frame size  
img_size = 224

img_size_tuple = (img_size, img_size)

# Number of channels (RGB)
num_channels = 3

# Flat frame size
img_size_flat = img_size * img_size * num_channels

# Number of classes for classification (Violence-No Violence)
num_classes = 2

# Number of files to train
_num_files_train = 1

# Number of frames per video
_images_per_file = 100

# Number of frames per training set
_num_images_train = _num_files_train * _images_per_file

# Video extension
video_exts = ".avi"

#### Helper-function for getting video frames
Function used to get 100 frames from a video file and convert the frame to a suitable format for the neural net.

In [5]:
def get_frames(current_dir, file_name):
    
    in_file = os.path.join(current_dir, file_name)
    
    images = []
    
    vidcap = cv2.VideoCapture(in_file)
    
    success,image = vidcap.read()
        
    count = 0

    while count<_images_per_file:
                
        RGB_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
        res = cv2.resize(RGB_img, dsize=(img_size, img_size),
                                 interpolation=cv2.INTER_CUBIC)
    
        images.append(res)
    
        success,image = vidcap.read()
    
        count += 1
        
    resul = np.array(images)
    
    resul = (resul / 255.).astype(np.float16)
        
    return resul

#### Helper function to get the names of the data downloaded and label it

In [6]:
def label_video_names(in_dir, sample_size=100):
    
    # list containing video names
    names = []
    # list containin video labels [1, 0] if it has violence and [0, 1] if not
    labels = []
    
    
    for current_dir, dir_names,file_names in os.walk(in_dir):
        
        for file_name in file_names:
            
            if file_name[0:2] == 'fi':
                labels.append([1,0])
                names.append(file_name)
            elif file_name[0:2] == 'no':
                labels.append([0,1])
                names.append(file_name)
                     
            
    c = list(zip(names,labels))
    # Suffle the data (names and labels)
    res = random.sample(c, sample_size)
    
    names, labels = zip(*res)
            
    return names, labels

#### Pre-Trained Model: VGG16

In [7]:
vid_model = VGG16(include_top=True, weights='imagenet')

553476096/553467096 [==============================] - 3s 0us/step


vid_model.summary()


We can observe the shape of the tensors expected as input by the pre-trained VGG16 model. In this case it is images of shape 224 x 224 x 3. Note that we have defined the frame size as 224x224x3. The video frame will be the input of the VGG16 net.

In [8]:
# We will use the output of the layer prior to the final
# classification-layer which is named fc2. This is a fully-connected (or dense) layer.
transfer_layer = vid_model.get_layer('fc2')

image_model_transfer = Model(inputs=vid_model.input,
                             outputs=transfer_layer.output)

transfer_values_size = K.int_shape(transfer_layer.output)[1]


print("The input of the VGG16 net have dimensions:",K.int_shape(vid_model.input)[1:3])

print("The output of the selecter layer of VGG16 net have dimensions: ", transfer_values_size)

The input of the VGG16 net have dimensions: (224, 224)
The output of the selecter layer of VGG16 net have dimensions:  4096


#### Generator that process one video through VGG16 each function call

In [ ]:
in_dir = '../RWF-2000/train'

### Functions to save transfer values from VGG16 to later use
We are going to define functions to get the transfer values from VGG16 with defined number of files. Then save the transfer values files used from training in one file and the ones uses for testing in another one. 

In [16]:
def process_transfer(vid_names, in_dir, labels):
    
    count = 0
    
    tam = len(vid_names)
    
    # Pre-allocate input-batch-array for images.
    shape = (_images_per_file,) + img_size_tuple + (3,)
    
    while count<tam:
        
        video_name = vid_names[count]
        
        image_batch = np.zeros(shape=shape, dtype=np.float16)
        
        try:
            image_batch = get_frames(in_dir, video_name)
        except Exception as e:
          print('Exception Error :', e, video_name)
          count += 1
          continue
        
        # Note that we use 16 bit floating pt to save memory
        shape = (_images_per_file, transfer_values_size)
        transfer_values = np.zeros(shape=shape, dtype=np.float16)
        
        transfer_values = image_model_transfer.predict(image_batch)
         
        labels1 = labels[count]
        
        aux = np.ones([100,2])
        
        labelss = labels1*aux
        
        yield transfer_values, labelss
        
        count+=1

In [17]:
def make_files(file_names, file_labels, output_file):
    
    gen = process_transfer(file_names, in_dir, file_labels)

    numer = 1
    n_files = len(file_names)

    # Read the first chunk to get the column dtypes
    chunk = next(gen)

    row_count = chunk[0].shape[0]
    row_count2 = chunk[1].shape[0]
    
    with h5py.File(output_file, 'w') as f:
    
        # Initialize a resizable dataset to hold the output
        maxshape = (None,) + chunk[0].shape[1:]
        maxshape2 = (None,) + chunk[1].shape[1:]
     
        dset = f.create_dataset('data', shape=chunk[0].shape, maxshape=maxshape,
                                chunks=chunk[0].shape, dtype=chunk[0].dtype)
    
        dset2 = f.create_dataset('labels', shape=chunk[1].shape, maxshape=maxshape2,
                                 chunks=chunk[1].shape, dtype=chunk[1].dtype)
    
         # Write the first chunk of rows
        dset[:] = chunk[0]
        dset2[:] = chunk[1]

        for chunk in gen:
            
            if numer == n_files:
            
                break

            # Resize the dataset to accommodate the next chunk of rows
            dset.resize(row_count + chunk[0].shape[0], axis=0)
            dset2.resize(row_count2 + chunk[1].shape[0], axis=0)

            # Write the next chunk
            dset[row_count:] = chunk[0]
            dset2[row_count:] = chunk[1]

            # Increment the row count
            row_count += chunk[0].shape[0]
            row_count2 += chunk[1].shape[0]
            
            print_progress(numer, n_files)
        
            numer += 1

#### Split the dataset into training set and test set

In [21]:
in_dir = "../keypoints/val"
fight, no_fight = [], []
for current_dir, dir_names, file_names in os.walk(in_dir):
  for file_name in file_names:
    if file_name[0:2] == 'fi':
      fight.append(file_name)
    else:
      no_fight.append(file_name)

print(len(fight), len(no_fight))

184 111


In [14]:
#tr_dir = '../RWF-2000/train'
#in_dir = "../opt_flow/train"
in_dir = "../keypoints/train"
names, labels = label_video_names(in_dir, sample_size=1108)

test_set = int(len(names)*0.2)
train_set = int(len(names)*0.8)

names_train = names[0:train_set]
names_test = names[train_set:]

labels_train = labels[0:train_set]
labels_test = labels[train_set:]

print(len(names_train), len(labels_train), len(names_test), len(labels_test))

886 886 222 222


In [18]:
make_files(names_train, labels_train, 'datasets/lstm_keypoints_train.h5')

- Progress: 99.9%

In [19]:
make_files(names_test, labels_test, 'datasets/lstm_keypoints_test.h5')

- Progress: 99.5%

In [22]:
# Process validation files
#val_dir = '../RWF-2000/val'
#in_dir = '../opt_flow/val'
in_dir = '../keypoints/val'
names_val, labels_val = label_video_names(in_dir, sample_size=295)
make_files(names_val, labels_val, 'datasets/lstm_keypoints_val.h5')
print(len(names_val), len(labels_val))

- Progress: 99.7%295 295
